In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# relative imports
from uncertainty_aware_diagnosis import (
    ICD10data,
    SimpleMLP,
    sklearnMLP,
)

# absolute imports
from torch.utils.data import DataLoader
from torch import nn
import torch
import numpy as np
from torchmetrics import F1Score, Recall, Accuracy

# paths
train_csv = "./data/train.csv"
val_csv = "./data/val.csv"
test_csv = "./data/test.csv"

In [ ]:
# variables
batch_size = 32
hidden_dim = 256
shuffle = True

num_epochs = 10
early_stopping_patience = 5
dropout = 0.2
learning_rate = 1e-3
scheduler_step_size = 5
scheduler_gamma = 0.5

In [ ]:
# load data
ds_train = ICD10data(train_csv)
ds_val = ICD10data(val_csv)
ds_test = ICD10data(test_csv)

train_loader = DataLoader(ds_train, batch_size=batch_size, shuffle=shuffle)
val_loader = DataLoader(ds_val, batch_size=batch_size, shuffle=shuffle)
test_loader = DataLoader(ds_test, batch_size=batch_size, shuffle=shuffle)

input_dim = ds_train.X.shape[1]
output_dim = ds_train.classes.shape[0]

print(f"(input_dim: {input_dim}, hidden_dim: {hidden_dim}, output_dim: {output_dim})")

In [ ]:
# define model
model = SimpleMLP(
    input_dim=input_dim, hidden_dim=hidden_dim, num_classes=output_dim, dropout=dropout
)
# loss function
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# scheduler = torch.optim.lr_scheduler.StepLR(
#     optimizer,
#     step_size=scheduler_step_size,
#     gamma=scheduler_gamma
# )
# metrics
f1_macro = F1Score(task="multiclass", average="macro", num_classes=output_dim)
recall_macro = Recall(task="multiclass", average="macro", num_classes=output_dim)

# counters
best_f1 = -np.inf
epochs_no_improve = 0

for epoch in range(num_epochs):
    # training loop
    model.train()
    running_loss = 0.0
    for batch_idx, (X, y) in enumerate(train_loader):
        optimizer.zero_grad()
        logits = model(X)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * X.size(0)
    avg_train_loss = running_loss / len(train_loader.dataset)

    # validation loop
    model.eval()
    val_loss = 0.0
    f1_macro.reset()
    recall_macro.reset()
    with torch.no_grad():
        for batch_idx, (X, y) in enumerate(val_loader):
            logits = model(X)
            val_loss += criterion(logits, y).item() * X.size(0)
    avg_val_loss = val_loss / len(val_loader.dataset)

    # calculate metrics
    y_pred = torch.argmax(logits, dim=1)
    f1_macro.update(y_pred, y)
    recall_macro.update(y_pred, y)

    # print progress
    print(
        f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, F1 Macro: {f1_macro.compute():.4f}, Recall Macro: {recall_macro.compute():.4f}"
    )

    # Step the learning rate scheduler
    # scheduler.step()  # (If using ReduceLROnPlateau: scheduler.step(val_macro_f1))

    # check for improvement for early stopping
    if f1_macro.compute() > best_f1:
        best_f1 = f1_macro.compute()
        best_model_state = model.state_dict()
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= early_stopping_patience:
            print(f"Early stopping triggered at epoch {epoch + 1}")
            break


if "best_model_state" in locals():
    # load best model
    model.load_state_dict(best_model_state)
    # save best model
    # torch.save(best_model_state, 'models/best_model.pth')

print(
    "=================================================================================\n"
)
print(f"Best F1 Macro: {best_f1:.4f}, Recall Macro: {recall_macro.compute():.4f}")

In [ ]:
# test best model
model.eval()
test_loss = 0.0
f1_macro.reset()
recall_macro.reset()
accuracy = Accuracy(task="multiclass", average="macro", num_classes=output_dim)

# Prepare a list to store predictions
predicted_probs = []

with torch.no_grad():
    for batch_idx, (X, y) in enumerate(test_loader):
        logits = model(X)
        loss = criterion(logits, y)
        test_loss += loss.item() * X.size(0)
        y_pred = torch.argmax(logits, dim=1)
        predicted_probs.append(logits)
        f1_macro.update(y_pred, y)
        recall_macro.update(y_pred, y)
        accuracy.update(y_pred, y)

# compute final values
test_acc = accuracy.compute()
test_f1 = f1_macro.compute()
test_recall = recall_macro.compute()
test_probs = torch.cat(predicted_probs)

print(f"Test Accuracy:          {test_acc:.4f}")
print(f"Test Macro-F1:          {test_f1:.4f}")
print(f"Test Macro-Recall:      {test_recall:.4f}")

# Calibration

In [ ]:
# Calibration imports
from pycalib.visualisations import plot_reliability_diagram
from sklearn.calibration import CalibratedClassifierCV
from pycalib.models.calibrators import LogisticCalibration
from pycalib.metrics import ECE, classwise_ECE, conf_ECE

In [ ]:
X_train = ds_train.X.cpu().numpy()
y_train = ds_train.y.cpu().numpy()
X_val = ds_val.X.cpu().numpy()
y_val = ds_val.y.cpu().numpy()
X_test = ds_test.X.cpu().numpy()
y_test = ds_test.y.cpu().numpy()

In [ ]:
# variables
num_epochs = 25
early_stopping_patience = 10
learn_rate = 1e-3
dropout = 0.2
hidden_dim = 256

model = SimpleMLP(
    input_dim=input_dim, hidden_dim=hidden_dim, num_classes=output_dim, dropout=dropout
)
model.fit(
    train_loader,
    val_loader,
    num_epochs=num_epochs,
    learning_rate=learning_rate,
    early_stopping_patience=early_stopping_patience,
    verbose=True,
)

y_pred = model.predict(ds_test.X)
y_proba = model.predict_proba(ds_test.X)

In [ ]:
_ = plot_reliability_diagram(
    y_test,
    [
        y_proba,
    ],
    legend=[
        "MLP",
    ],
    class_names=list(ds_test.classes),
)

In [ ]:
# 1) get your raw logits from the MLP
props_val = model.predict_proba(ds_val.X)  # shape (n_val, n_classes)
probs_test = model.predict_proba(ds_test.X)  # shape (n_test, n_classes)

# 2) instantiate & fit the pycalib logistic (Platt) calibrator
calibrator = LogisticCalibration(
    C=0.002, solver="lbfgs", multi_class="multinomial", log_transform=True
)
calibrator.fit(props_val, y_val)

# 3) use it to get calibrated probabilities on your test set
probs_calibrated = calibrator.predict_proba(probs_test)  # shape (n_test, n_classes)

_ = plot_reliability_diagram(
    ds_test.y.numpy(),
    [probs_test, probs_calibrated],
    legend=["MLP", "+ Calibrator"],
    class_names=list(ds_test.classes),
)

for metric in ECE, conf_ECE, classwise_ECE:
    print(metric.__name__)
    print("Classifier = {:.3f}".format(metric(ds_test.y.numpy(), probs_test, bins=15)))
    print(
        "Calibrator = {:.3f}".format(
            metric(ds_test.y.numpy(), probs_calibrated, bins=15)
        )
    )
    print("")

[explains ECE and nessesary binning](https://towardsdatascience.com/expected-calibration-error-ece-a-step-by-step-visual-explanation-with-python-code-c3e9aa12937d/)

In [ ]:
# Convert calibrated probabilities to predicted classes
y_pred_calibrated = torch.argmax(torch.tensor(probs_calibrated), dim=1)

# Initialize metrics
f1_macro = F1Score(task="multiclass", average="macro", num_classes=len(ds_test.classes))
recall_macro = Recall(
    task="multiclass", average="macro", num_classes=len(ds_test.classes)
)

# Update metrics with predictions and true labels
f1_macro.update(y_pred_calibrated, ds_test.y)
recall_macro.update(y_pred_calibrated, ds_test.y)

# Compute final values
final_f1 = f1_macro.compute()
final_recall = recall_macro.compute()

# Print results
print(f"F1 Macro: {final_f1:.4f}")
print(f"Recall Macro: {final_recall:.4f}")

In [ ]:
print(probs_test[0])
print(probs_calibrated[0])

### How to get confidence interval from point estimation per class?

In [ ]:
cal = CalibratedClassifierCV(estimator=model, method="sigmoid", cv="prefit")
cal.fit(ds_val.X.numpy(), ds_val.y.numpy())

scores_cal = cal.predict_proba(ds_test.X)

_ = plot_reliability_diagram(
    ds_test.y.numpy(),
    [test_probs.numpy(), scores_cal],
    legend=["MLP", "+ Calibrator"],
    class_names=list(ds_test.classes),
)

In [ ]:
# 1) pull out your arrays from the Dataset
#    (they’re already torch.Tensors of shape [n_samples, n_features])
X_train = ds_train.X.cpu().numpy()
y_train = ds_train.y.cpu().numpy()
X_val = ds_val.X.cpu().numpy()
y_val = ds_val.y.cpu().numpy()
X_test = ds_test.X.cpu().numpy()
y_test = ds_test.y.cpu().numpy()

# 2) instantiate the sklearn‐wrapped MLP
mlp = sklearnMLP(
    input_dim=X_train.shape[1],  # number of columns in X_train
    hidden_dim=128,  # pick whatever you like
    num_classes=len(ds_train.classes),  # your LabelEncoder’s classes
    batch_size=64,
    num_epochs=20,
    learning_rate=1e-3,
    early_stopping_patience=5,
    device="cpu",  # or "cuda" if you like
    verbose=True,
)

# 3) fit on your training set
mlp.fit(X_train, y_train)

In [ ]:
# 4) calibrate on the validation set
cal = CalibratedClassifierCV(estimator=mlp, method="sigmoid", cv="prefit")
cal.fit(X_val, y_val)

# 5) now you can predict or get calibrated probabilities
probs_test_uncal = mlp.predict_proba(X_test)  # raw MLP probs
probs_test_cal = cal.predict_proba(X_test)  # after sigmoid‐calibration

In [ ]:
from uncertainty_aware_diagnosis import SklearnWrapper

In [ ]:
# pull out your arrays
X_train, y_train = ds_train.X.numpy(), ds_train.y.numpy()
X_val, y_val = ds_val.X.numpy(), ds_val.y.numpy()
X_test, y_test = ds_test.X.numpy(), ds_test.y.numpy()

# wrap it
sk_mlp = SklearnWrapper(
    input_dim=X_train.shape[1],
    hidden_dim=128,
    num_classes=len(ds_train.classes),
    batch_size=64,
    num_epochs=20,
    learning_rate=1e-3,
    device="cpu",  # or "cuda"
    verbose=True,
)

# 1) fit
sk_mlp.fit(X_train, y_train)

In [ ]:
# 2) calibrate
cal = CalibratedClassifierCV(sk_mlp, method="sigmoid", cv="prefit")
cal.fit(X_val, y_val)

# 3) now you can do
probs_uncal = sk_mlp.predict_proba(X_test)
probs_cal = cal.predict_proba(X_test)

In [ ]:
_ = plot_reliability_diagram(
    ds_test.y.numpy(),
    [probs_test_uncal, probs_test_cal],
    legend=["MLP", "+ Calibrator"],
    class_names=list(ds_test.classes),
)